In [1]:
import numpy as np

In [2]:
F_info=np.load("learned_ndv_estimator/model_training/training_data/rfs_F_infos.npy")

In [3]:
f_s=np.load("learned_ndv_estimator/model_training/training_data/rfs_f_s.npy",allow_pickle=True)

In [4]:
import numpy as np
from numpy.polynomial import Chebyshev
from numpy.polynomial import Hermite
from scipy import optimize
from scipy.special import comb
from estndv import ndvEstimator

In [5]:
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import style
#plt.rcParams['font.sans-serif']=['simhei']
matplotlib.rcParams['text.usetex'] = True
from collections import Counter
import scipy

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from scipy.special import comb
from scipy import interpolate
import scipy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PolyEntropyNet(nn.Module):
    def __init__(self,L=7,H=200):
        super(PolyEntropyNet,self).__init__()
        self.L = L
        self.H = H
        self.weight = Parameter(torch.Tensor(self.H))
        self.weight.requires_grad = True
        self.fc1 = nn.Linear(self.L*10, self.L*10//2)
        self.fc2 = nn.Linear(self.L*10//2, self.L)
        self.fc3 = nn.Linear(self.L, self.L)
        self.fc1.requires_grad = True
        self.fc2.requires_grad = True
        self.fc3.requires_grad = True
        self.i = torch.Tensor( [ i+1 for i in range(self.H) ] ).float().to(device)
    def set_paras(self):
        self.weight.data = torch.Tensor([1.0 for i in range(self.H)])
    def forward(self,fi,M_tensor,N,n):
        bt_temp= F.leaky_relu(self.fc1( torch.log(fi+1) ) )
        bt=F.leaky_relu(self.fc2(bt_temp))
        temp = torch.einsum('ij,ijk->ik', bt,M_tensor)
        pi=(self.i.reshape(self.H,1)/N.reshape(1,N.shape[0])).t()
        temp=(( temp+pi*torch.log(pi)).abs()*(self.weight/self.weight.sum()).reshape(self.weight.shape[0],1)).sum(1)
        f0 = F.relu(self.fc3( bt ).sum(1) )
        return f0,temp
    
penet = PolyEntropyNet().to(device)
penet.set_paras()
for param in penet.parameters():
    param.data = param.data.to(device)

optimizer = torch.optim.Adam(penet.parameters(),lr=0.001)

# data prepare
fea = torch.tensor([[0.0 for j in range(70)]  for i in range(f_s.shape[0])]).to(device)
entropy_plus=[]
for i in range(f_s.shape[0]):
    f_s_n=sum(f_s[i][:,0]*f_s[i][:,1])
    entropy_p=0.0
    for item in f_s[i]:
        if item[0]>70:
            entropy_p+= item[1]*item[0]/f_s_n*np.log(f_s_n/item[0])#+1.0/f_s_n
        else:
            fea[i][item[0]-1]=item[1]
            entropy_p+= item[1]*item[0]/f_s_n*np.log(f_s_n/item[0])#+1.0/f_s_n
    entropy_plus.append(entropy_p)
entropy_plus = torch.Tensor(entropy_plus).to(device)

d = torch.tensor([ sum(f[:,1]) for f in f_s]).to(device)
n = torch.tensor([ sum(f[:,0]*f[:,1]) for f in f_s] ).to(device)
N = torch.tensor(F_info[:,0]).to(device)
Entro = torch.tensor( F_info[:,2]).to(device)

M_tensor=[]
size = F_info.shape[0]
L=7
H=200
data_n=np.array([sum(te[:,1]) for te in f_s])
data_N=F_info[:,0]
for s in range(size):
    M=[]
    for t in range(1,L+1):
        M_temp=[]
        for j in range(1,H+1):
            M_temp.append( comb(data_n[s],t)*(j/(data_N[s]-j))**t)
        M.append(M_temp)
    M=np.array(M)
    M[M<0.001]=0
    M_tensor.append(M)    
M_tensor = torch.tensor(M_tensor).float().to(device)

def lossfun_entropy(y,y_pred,l0_loss):
    l=torch.mean(  torch.square(y_pred-y) + l0_loss.abs() )
    return l

batch_size=200
train_loss=[]
for i in range(300):
    print("epoch",i)
    for epoch in range(200):
        predict,l0_loss=penet(
            fea[epoch*batch_size:((epoch+1)*batch_size)] , 
            M_tensor[epoch*batch_size:((epoch+1)*batch_size)],
            N[epoch*batch_size:((epoch+1)*batch_size)],
            n[epoch*batch_size:((epoch+1)*batch_size)])
        y=torch.relu(Entro-entropy_plus)
        loss=lossfun_entropy(y[epoch*batch_size:((epoch+1)*batch_size)],predict,
                             l0_loss)
        loss.requires_grad_(True)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss.append(loss)

/home/jiajun_li/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)


epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99
epoch 100
epoch 101
epoch 102
epoch 103
epoch 104
epoch 105
epoch 106
epoch 107
epoch 108
epoch 109
epoch 110


In [9]:
def Fi2fi(Fi,q):
    fs = []
    for item in Fi:
        f_i=np.random.binomial(item[0],q,size=item[1])
        fs.append(f_i)
    fs = np.concatenate(fs)
    fs = fs[fs>0]
    unique, counts = np.unique(fs,return_counts=True)
    f_sparse = np.array([unique,counts]).T
    return f_sparse
renEsti=ndvEstimator()
import os
Dir="kasandr"
q=0.001
kasResult=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        r5.append( np.abs(temp-En) )   
    kasResult.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

kasResult=np.array(kasResult)
for i in range(kasResult.shape[1]):
    print(np.mean(kasResult[:,i]))

1.1494325524199114
0.9729270963998351
19.06044869781375
0.45239484097276417


In [10]:
import os
Dir="kasandr"
q=0.005
kasResult=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        r5.append( np.abs(temp-En) )
        
    kasResult.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

kasResult=np.array(kasResult)
for i in range(kasResult.shape[1]):
    print(np.mean(kasResult[:,i]))

0.6522908480608957
0.5056012315676519
3.7746788642610976
0.28172469564846586


In [11]:
import os
Dir="kasandr"
q=0.01
kasResult=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        r5.append( np.abs(temp-En) )
        
    kasResult.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

kasResult=np.array(kasResult)
for i in range(kasResult.shape[1]):
    print(np.mean(kasResult[:,i]))

0.4753002368141702
0.3466162879613011
1.8904896427993687
0.21500141492911748


In [12]:
import os
Dir="Airlines"
q=0.001
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )

        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.02762446863182404
0.010060569487000381
20.500976415872174
0.027624452114105226


In [13]:
import os
Dir="Airlines"
q=0.005
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        
        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.006507923906911151
0.0024870071460573184
4.090577266367129
0.006507933139801025


In [14]:
import os
Dir="Airlines"
q=0.01
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        #print(temp)
        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.003653386469935449
0.00176315590363747
2.043636135477109
0.0036533832550048827


In [15]:
def SVD_entropy(fi,q,N=10**6):
    M=[]
    for j in range(10,200):
        M_temp=[]
        for t in range(1,10):
            M_temp.append( comb(N*q,t)*(j/(N-j))**t )
        M.append(M_temp)
    M=np.array(M)
    M[M<0.01]=0
    u,s,v = np.linalg.svd(M,full_matrices=False)
    s_inv=( np.array( [ (i+1)/N*np.log(N/(i+1)) for i in range(s.shape[0]) ] )  /s)
    s_inv[np.isinf(s_inv)] = 0
    inv = np.matmul(v.T*s_inv , u.T )
    bi=[]
    for i in range(9):
        #print(sum(inv[i,:]))
        bi.append(sum(inv[i,:]))
    result=0
    for x in fi:
        if x[0]<10:
            result+=x[1]*bi[x[0]-1]
    if result>0:
        return result+sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )#sum(fi[:,1])
    else:
        return sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )#sum(fi[:,1])
def Fi2fi(Fi,q):
    fs = []
    for item in Fi:
        f_i=np.random.binomial(item[0],q,size=item[1])
        fs.append(f_i)
    fs = np.concatenate(fs)
    fs = fs[fs>0]
    unique, counts = np.unique(fs,return_counts=True)
    f_sparse = np.array([unique,counts]).T
    return f_sparse
renEsti=ndvEstimator()
import os
Dir="SSB"
q=0.001
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )

        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

1.5018011285061659
1.2928135443675135
17.260647106108873
0.14898082789252787


In [16]:
import os
Dir="SSB"
q=0.005
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        #print(temp)
        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.9015412751562009
0.7231101312219057
3.3651043361863358
0.0532607541364782


In [17]:
import os
Dir="SSB"
q=0.01
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )

        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.6792526787289997
0.5176179112817797
1.6784082447383017
0.036526027847738826


In [18]:
import os
Dir="NCVoter"
q=0.001
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        
        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.5250490632133745
0.45866272453639106
21.707687820522725
0.2721954326753662


In [19]:
import os
Dir="NCVoter"
q=0.005
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )

        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.36006606383667095
0.31423977609826736
4.1941368200135605
0.2046379915784559


In [20]:
import os
Dir="NCVoter"
q=0.01
Result=[]
order_D = []
En_list=[]
for filename in os.listdir(Dir):
    pathname=os.path.join(Dir,filename)
    Fi=np.loadtxt(pathname,delimiter=",",dtype=int)
    if len(Fi.shape)==1:
        Fi=np.array([Fi])
    r1=[]
    r2=[]
    r3=[]
    r4=[]
    r5=[]
    r6=[]
    r7=[]
    order_D.append(sum(Fi[:,1]))
    En_list.append( sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) ) )
    for r in range(5):
        fii=Fi2fi(Fi,q)
        fi_temp=fii
        max_i=max(max(fi_temp[:,0]),L)
        fi_dict={ item[0]:item[1] for item in fi_temp }
        fi=[]
        n=0
        for i in range(1,max_i+1):
            if i in fi_dict.keys():
                fi.append([i,fi_dict[i]])
                n+=i*fi_dict[i]
            else:
                fi.append([i,0])
        fi=np.array(fi)
        
        N=sum(Fi[:,0]*Fi[:,1])
        En=sum( Fi[:,1]*Fi[:,0]/sum(Fi[:,1]*Fi[:,0])*np.log( sum(Fi[:,1]*Fi[:,0])/Fi[:,0] ) )
        from entropy import *
        entro_estimator = Entropy(N)
        temp=entro_estimator.estimate_plug( fii )*np.log(2)
        r1.append( np.abs(temp-En) )
        temp=entro_estimator.estimate_Miller_Madow( fii )*np.log(2)
        r2.append( np.abs(temp-En) )
        temp=entro_estimator.estimate(fii)*np.log(2)
        r3.append( np.abs(temp-En) )
        
        
        L=7
        H=200
        data_n=n
        data_N=N
        M=[]
        for t in range(1,L+1):
            M_temp=[]
            for j in range(1,H+1):
                M_temp.append( comb(n,t)*(j/(N-j))**t)
            M.append(M_temp)
        M=np.array(M)
        M[M<0.001]=0
        feat=[0.0 for i in range(70)]
        for item in fii:
            if item[0]>70:
                continue
            else:
                feat[item[0]-1]=item[1]
        temp=penet(torch.tensor([feat]).float().to(device),
                  torch.tensor([M]).float().to(device),
                  torch.tensor([N]).float().to(device),
                  torch.tensor([n]).float().to(device),
                 )[0].cpu().detach().numpy()+ sum( fi[:,1]*fi[:,0]/sum(fi[:,1]*fi[:,0])*np.log( sum(fi[:,1]*fi[:,0])/fi[:,0] ) )
        
        r5.append( np.abs(temp-En) )
        
    Result.append([np.median(r1),
                  np.median(r2), 
                  np.median(r3),
                  np.median(r5)])
        

Result=np.array(Result)
for i in range(Result.shape[1]):
    print(np.mean(Result[:,i]))

0.29943270476430606
0.2594498012301002
2.068844398940328
0.18806978335364682
